In [5]:
from pytesseract import image_to_data, Output
import cv2
from tqdm import tqdm
import pandas as pd
import os
import pickle

In [6]:
OCR_DATA_PATH  = './../../data/ocr/docbank/images_processed/'
TXT_DATA_PATH  = './../../results/ocr/tesseract/'
image_data_dir = os.listdir(OCR_DATA_PATH)

In [3]:
results = {}

for image_file in tqdm(image_data_dir):
    predictions = []
    image = cv2.imread(OCR_DATA_PATH + image_file)
    d = image_to_data(image, output_type=Output.DICT)
    for i in range(len(d['level'])):
        if d['level'][i]==5:
            values = []
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            (x, y, w, h) = (int(x), int(y), int(w), int(h))

            values.append(d['block_num'][i])
            values.append(d['par_num'][i])
            values.append(d['line_num'][i])
            values.append(d['word_num'][i])
            values.append(d['conf'][i])
            values.append(d['left'][i])
            values.append(d['top'][i])
            values.append(d['left'][i] + d['width'][i])
            values.append(d['top'][i] + d['height'][i])
            values.append(d['text'][i])
            predictions.append(values)
    results[image_file] = predictions
    
    
with open(TXT_DATA_PATH + 'results_processed.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)

100%|██████████| 100/100 [03:44<00:00,  2.25s/it]


In [7]:
with open(TXT_DATA_PATH + 'results_processed.pkl', 'rb') as f:
    results = pickle.load(f)

In [10]:
for image,result in tqdm(results.items()):
    name = image[:len(image) - 4]
    df = pd.DataFrame(result, columns = ['block', 'para', 'line', 'word','confidence', 'X1', 'Y1', 'X2', 'Y2', 'token'])
    df.to_csv(TXT_DATA_PATH + 'txt_processed/' +  name + '.txt', sep=' ',index=False)

100%|██████████| 100/100 [00:00<00:00, 428.35it/s]
